In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier, CatBoostRegressor, Pool
# from plot_metric.functions import BinaryClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss, roc_auc_score
import seaborn as sns
import plotly.express as px

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default
from google.cloud import bigquery
client = bigquery.Client(project='dev-sd-lake')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
sales_prob_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/sales_prob_price_model.pkl")
domestic_export_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/domestic_export_price_model.pkl")
time_at_a_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/time_at_a_price_model.pkl")
amount_at_a_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/amount_at_a_price_model.pkl")
X_val_sales_prob = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_sales_prob.pkl")
X_val_domestic_export = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_domestic_export.pkl")

# default 1,2,3,4 for fern

In [ ]:
test_data = X_val_sales_prob.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'time', 'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd
17816,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416


In [ ]:
def get_dummy_prices(x):
  return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop = True)
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']
test_data = test_data.drop('new_price_per_unit_usd', axis = 1)

In [ ]:
test_data['probs'] = sales_prob_price_model.predict_proba(test_data)[:,1]
test_data['probs'] = test_data['probs'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) / test_data['asking_price_per_unit_usd'] * 100

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,order_price_per_unit_usd,probs,discount / price_hike_from_asking
0,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.2912,81.221640,-30.0
1,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.3328,76.753807,-20.0
2,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.3744,78.555181,-10.0
3,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.416,67.737411,0.0
4,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.4576,59.062756,10.0
5,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.4992,56.564496,20.0
6,SYCBB018-60,garnier,skin_care,body_lotions_&_creams,loin,1.0,280.0,NaN,excess,1.71,0.416,0.5408,54.987902,30.0


# default 5,6

In [ ]:
test_data = X_val_domestic_export.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'time', 'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd', 'domestic_export']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export
421500,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic


In [ ]:
def get_dummy_prices(x):
  return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop = True)
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']
test_data = test_data.drop('new_price_per_unit_usd', axis = 1)

In [ ]:
test_data['probs'] = domestic_export_price_model.predict_proba(test_data)[:,1]
test_data['probs'] = test_data['probs'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) / test_data['asking_price_per_unit_usd'] * 100

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export,order_price_per_unit_usd,probs,discount / price_hike_from_asking
0,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,18.341233,36.546140,-30.0
1,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,20.961409,96.005462,-20.0
2,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,23.581585,90.886952,-10.0
3,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,26.201761,96.895372,0.0
4,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,28.821938,21.368473,10.0
5,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,31.442114,7.661917,20.0
6,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,34.06229,3.916800,30.0


# default 7,8,9,10

In [ ]:
X_val[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd', 'domestic_export', 'sellability']]

KeyError: "['sellability'] not in index"